In [ ]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
import posixpath
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

In [ ]:
# download command
def dx_download(project_name, file_name, dest_name):
    quoted_path = f'"{project_name}:{file_name}"'
    download_command = f"dx download {quoted_path} -o {dest_name}"
    #print(f"Downloading {file_name} from {project_name} to {dest_name}")
    subprocess.run(download_command, shell=True, check=True)

# upload command
def dx_upload(file_name, dest_name):
    upload_command = f"dx upload {file_name} -o {dest_name}"
    #print(f"Uploading {file_name} to {dest_name}")
    subprocess.run(upload_command, shell=True, check=True)

# directory list command
def dx_dir_list(project_name, folder_name):
    dx_path = f"{project_name}:{folder_name}"
    try:
        result = subprocess.run(
            ["dx", "ls", dx_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True,
            text=True
        )
        return result.stdout.strip().split('\n')
    except subprocess.CalledProcessError as e:
        print(f"Failed to list {dx_path}: {e.stderr}")
        return []

## Preparation

In [ ]:
# project name    
# project_id = dxpy.api.user_get_project({'project': dxpy.DXProject('').get_id()})['project']
# project_name = dxpy.DXProject(project_id).describe()['name']
project_name = "project-GxqpVq0Jpp5Py82xVbZV198y"

# cloud genotype data path
genotype_origin_folder = "/GWAS_pipeline"


# Create a directory for storing the downloaded files
In_saving_folder = "pheno"
if not os.path.exists(In_saving_folder):
    os.makedirs(In_saving_folder)

## Collect data

In [ ]:
## Loop to collect all pheno files for all cohort images
# Single file download task
def collect_single_file(project_name, full_path, dest_path):
    if os.path.exists(dest_path):
        return 
    try:
        dx_download(project_name, full_path, dest_path)
    except Exception as e:
        print(f"Failed to download {full_path}: {e}")

def collect_pheno(project_name, folder_name, dest_folder, max_workers=12):
    # Check all folders (10~59, will be update)
    subfolder_list = dx_dir_list(project_name, folder_name)

    # Creat task
    tasks = []

    for subfolder in subfolder_list:
        subfolder_path = posixpath.join(folder_name, subfolder)
        # Check all files in the subfolder
        subfolder_files = dx_dir_list(project_name, subfolder_path)
        for file in subfolder_files:
            if file.endswith(("Template_lh.csv", "Template_rh.csv")):
                full_path = posixpath.join(subfolder_path, file)
                dest_path = os.path.join(dest_folder, file)
                # Make it a task
                tasks.append((project_name, full_path, dest_path))

    # Run in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(collect_single_file, *task) for task in tasks]
        concurrent.futures.wait(futures)
        
        
# Collect the phenotype files
phenotype_folder= "Test/Measurement_template"
dest_folder = In_saving_folder
collect_pheno(project_name, phenotype_folder, dest_folder, max_workers=16)


## Read data

In [ ]:
## Read the phenotype files
# Read function return target format
def read_pheno(file):
    df = pd.read_csv(file, index_col=0)

    df.index = df.index.astype(str)

    # Extract the file name info
    filename = os.path.basename(file)
    parts = filename.split("_")

    try:
        subject_id = parts[1]
        version = f"{parts[3]}_{parts[4]}"
    except IndexError:
        raise ValueError(f"Filename {filename} is malformed.")
    fid = iid = subject_id
    
    # Remove medial wall
    if "-1" in df.columns:
        df = df.drop(columns=["-1"])
    # Add prefix to all remaining columns based on file name
    prefix = "lh" if "lh" in file else "rh"

    # Reformat the dataframe
    # Extract per-vertex features (1~12)
    vertices = [col for col in df.columns if col.isdigit()]
    
    out = {}

    for parameter in df.index:
        if parameter.endswith("_avg"):
            base = parameter.replace("_avg", "")
            out.update({
                f"{prefix}_{base}_{i}": df.loc[parameter, v]
                for i, v in enumerate(vertices, start=1)
            })

    # Add summary features
    if "area_sum" in df.index:
        out[f"{prefix}_area_total"] = df.loc["area_sum", vertices].sum()
    if "thickness_sum" in df.index:
        out[f"{prefix}_thickness_mean"] = df.loc["thickness_sum", vertices].mean()
    if "volume_sum" in df.index:
        out[f"{prefix}_volume_total"] = df.loc["volume_sum", vertices].sum()

    # create the final DataFrame
    out = {
        "FID": fid,
        "IID": iid,
        "version": version,
        **out  # unsure, to be update
    }

    return pd.DataFrame(out, index=[0])



# Make it parallel
def parallel_read(files, max_workers=8):
    results = []
    total = len(files)
    completed = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(read_pheno, f): f for f in files}
        for future in as_completed(future_to_file):
            completed += 1
            try:
                df = future.result()
                results.append(df)
                print(f"processed {completed}/{total} files")
            except Exception as e:
                print(f"Error reading {future_to_file[future]}: {e}")
    return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

# merge function for lh and rh data
def merge_lh_rh(lh_files, rh_files, max_workers=8):
    lh_df = parallel_read(lh_files, max_workers) if lh_files else pd.DataFrame()
    rh_df = parallel_read(rh_files, max_workers) if rh_files else pd.DataFrame()

    merged = pd.merge(lh_df, rh_df, on=["FID", "IID", "version"], how="outer")

    id_cols = ["FID", "IID", "version"]
    lh_cols = sorted([col for col in merged.columns if col.startswith("lh_")])
    rh_cols = sorted([col for col in merged.columns if col.startswith("rh_")])
    final_cols = id_cols + lh_cols + rh_cols
    return merged[final_cols].sort_values("IID")

In [ ]:
import glob

lh_files = glob.glob("pheno/*_Template_lh.csv")
rh_files = glob.glob("pheno/*_Template_rh.csv")

merged_df = merge_lh_rh(lh_files, rh_files)
print(merged_df.head())

In [ ]:
# Upload the merged file
merged_file_name = "merged_pheno.csv"
merged_df.to_csv(merged_file_name, index=False)
# Upload the merged file to the project
file_name = "merged_pheno.csv"
dest_name = f"{genotype_origin_folder}/Image_dataset/merged_pheno.csv"
dx_upload(file_name, dest_name)

## QC: remove the outline

## Update subject list

In [ ]:
# Select required columns from merged phenotype data
merged_pheno = merged_df
selected_columns = ["FID", "IID", "version"]
lh_area_cols = [col for col in merged_pheno.columns if col.startswith("lh_area_")]
rh_area_cols = [col for col in merged_pheno.columns if col.startswith("rh_area_")]
lh_thick_cols = [col for col in merged_pheno.columns if col.startswith("lh_thickness_")]
rh_thick_cols = [col for col in merged_pheno.columns if col.startswith("rh_thickness_")]
volume_cols = [col for col in merged_pheno.columns if col.endswith("volume_total")]

final_selected_columns = selected_columns + lh_area_cols + rh_area_cols + lh_thick_cols + rh_thick_cols + volume_cols
# Select the columns from the merged phenotype data
select_pheno = merged_pheno[final_selected_columns]
print(select_pheno.head())

In [ ]:
# Create a final pheno data for mergeing lh and rh
merged_df = select_pheno.copy()
# Collect matching column pairs
lh_cols = [col for col in merged_df.columns if col.startswith("lh_") and col.replace("lh_", "rh_") in merged_df.columns]
# Create average columns
for lh_col in lh_cols:
    rh_col = lh_col.replace("lh_", "rh_")
    
    if "thickness" in lh_col:
        mean_col = lh_col.replace("lh_", "")
        merged_df[mean_col] = merged_df[[lh_col, rh_col]].mean(axis=1)
    
    elif "area" in lh_col:
        sum_col = lh_col.replace("lh_", "")
        merged_df[sum_col] = merged_df[[lh_col, rh_col]].mean(axis=1)

In [ ]:
# Drop all columns starting with 'lh_' or 'rh_'
cols_to_drop = [col for col in merged_df.columns if col.startswith("lh_") or col.startswith("rh_")]
final_pheno = merged_df.drop(columns=cols_to_drop)
# print(final_pheno.head())
# print(list(final_pheno.columns))


# Save the final phenotype data
final_pheno.to_csv('final_pheno.csv', index=False)

In [ ]:
import re
# Clean the column names
id_cols = ['FID', 'IID', 'version']
# get all area and thickness region columns, excluding totals
area_cols = sorted(
    [col for col in final_pheno.columns if re.match(r'area_\d+$', col)],
    key=lambda x: int(x.split('_')[1])
)
thickness_cols = sorted(
    [col for col in final_pheno.columns if re.match(r'thickness_\d+$', col)],
    key=lambda x: int(x.split('_')[1])
)
# Rename the total columns
final_pheno_cleaned = final_pheno.rename(columns={
    'area_total': 'total_SA',
    'thickness_mean': 'mean_CT'
})
# Append the renamed summary columns at the end
summary_cols = ['total_SA', 'mean_CT']
# Final column order
final_cols_order = id_cols + area_cols + thickness_cols + summary_cols
final_pheno_cleaned = final_pheno_cleaned[final_cols_order]
# Save the cleaned final phenotype data
final_pheno_cleaned.to_csv('final_pheno_cleaned.csv', index=False)

## Filter subject list

In [ ]:
# Count longitutinal participants

version_counts = final_pheno_cleaned.groupby('IID')['version'].unique()
revisit_counts = version_counts[version_counts.apply(lambda x: set(x) >= {'2_0', '3_0'})].index
# Filter the phenotype data to remove participants with version 3_0 (keep only 2_0)
final_pheno_subset = final_pheno_cleaned[final_pheno_cleaned['version'] != '3_0']

print(f"Participants with both 2_0 and 3_0: {len(revisit_counts)}")
print(f"Original entries: {len(final_pheno_cleaned)}")
print(f"Remaining entries after removing 3_0: {len(final_pheno_subset)}")

In [ ]:
# filter the total surface area
values = final_pheno_subset['total_SA']
mean = values.mean()
std = values.std()
# Check num of outliers
outliers = values[(values < mean - 3 * std) | (values > mean + 3 * std)]
# print(f"Number of outliers: {len(outliers)}")


# Remove outliers
final_pheno_qced = final_pheno_subset[~final_pheno_subset['total_SA'].isin(outliers)]
# print(f"Remaining entries after removing outliers: {len(final_pheno_qced)}")


# filter the total thickness
values = final_pheno_qced['mean_CT']
mean = values.mean()
std = values.std()
# Check num of outliers
outliers = values[(values < mean - 3 * std) | (values > mean + 3 * std)]
# print(f"Number of outliers: {len(outliers)}")

# Remove outliers
final_pheno_qced = final_pheno_qced[~final_pheno_qced['mean_CT'].isin(outliers)]
# print(f"Remaining entries after removing outliers: {len(final_pheno_qced)}")

### plot show

In [ ]:
# ### plot show

# import matplotlib.pyplot as plt
# from scipy.stats import norm

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(values, bins=100, density=True, alpha=0.6, color='blue', edgecolor='black')

# # Normal distribution curve
# x = np.linspace(values.min(), values.max(), 1000)
# pdf = norm.pdf(x, mean, std)
# plt.plot(x, pdf, 'r--', label='Normal Distribution')

# # Add outlier thresholds
# plt.axvline(mean - 3 * std, color='red', linestyle='dashed', linewidth=1, label='-3 SD')
# plt.axvline(mean + 3 * std, color='red', linestyle='dashed', linewidth=1, label='+3 SD')

# plt.title('Histogram of Total Surface Area with Normal Curve')
# plt.xlabel('Total Surface Area')
# plt.ylabel('Density')
# plt.grid(axis='y', alpha=0.75)
# plt.legend()
# plt.show()
# # Remove outliers
# final_pheno_qced = final_pheno_subset[~final_pheno_subset['total_SA'].isin(outliers)]
# print(f"Remaining entries after removing outliers: {len(final_pheno_qced)}")


# import matplotlib.pyplot as plt
# from scipy.stats import norm
# # plot histogram of the total surface area
# values = final_pheno_qced['mean_CT']
# mean = values.mean()
# std = values.std()
# # Check num of outliers
# outliers = values[(values < mean - 3 * std) | (values > mean + 3 * std)]
# print(f"Number of outliers: {len(outliers)}")
# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(values, bins=100, density=True, alpha=0.6, color='blue', edgecolor='black')

# # Normal distribution curve
# x = np.linspace(values.min(), values.max(), 1000)
# pdf = norm.pdf(x, mean, std)
# plt.plot(x, pdf, 'r--', label='Normal Distribution')

# # Add outlier thresholds
# plt.axvline(mean - 3 * std, color='red', linestyle='dashed', linewidth=1, label='-3 SD')
# plt.axvline(mean + 3 * std, color='red', linestyle='dashed', linewidth=1, label='+3 SD')

# plt.title('Histogram of Mean Thickness with Normal Curve')
# plt.xlabel('Total Surface Area')
# plt.ylabel('Density')
# plt.grid(axis='y', alpha=0.75)
# plt.legend()
# plt.show()
# # Remove outliers
# final_pheno_qced = final_pheno_qced[~final_pheno_qced['mean_CT'].isin(outliers)]
# print(f"Remaining entries after removing outliers: {len(final_pheno_qced)}")

## Update all necesarry list

In [ ]:
# Download the full participant data

dx_download(project_name, "full_participant.csv", "full_participant.csv")

demographic_data = pd.read_csv('full_participant.csv')

In [ ]:
# Filter for Caucasian in demographic data
caucasian_participants = demographic_data[demographic_data['p22006'] == 1.0]
num_caucasian = (demographic_data['p22006'] == 1.0).sum()
print(f"Number of Caucasian participants: {num_caucasian}")

total_participants = len(demographic_data)
print(f"Total participants: {total_participants}")
print(f"Proportion Caucasian: {100 * num_caucasian / total_participants:.2f}%")

# Check which participants in final_pheno_qced are in the Caucasian list
caucasian_ids = set(caucasian_participants['eid'])
matched_ids = final_pheno_qced[final_pheno_qced['IID'].isin(caucasian_ids)]

print(f"Total participants in final_pheno_qced: {len(final_pheno_qced)}")
print(f"Number of Caucasian participants matched: {len(matched_ids)}")
print(f"Percentage matched: {100 * len(matched_ids) / len(final_pheno_qced):.2f}%")

# Save the final phenotype data for Caucasian participants
final_pheno_caucasian = final_pheno_qced[final_pheno_qced['IID'].isin(caucasian_ids)]
print(f"Final number of Caucasian participants: {len(final_pheno_caucasian)}")

# Save the cleaned final phenotype data
final_pheno_caucasian.to_csv('final_pheno_caucasian.csv', index=False)

In [ ]:
# Generate demographic data for fastGWA
demographic_data = demographic_data.drop(columns=["p22006"])
# GCTA format
demographic_data.insert(0, "FID", demographic_data["eid"])
demographic_data.insert(1, "IID", demographic_data["eid"])
# categorical variables
covar_cols = ["FID", "IID", "p31", "p54_i2"]
demographic_data[covar_cols].to_csv("ukb_covar.txt", sep="\t", index=False, na_rep="NA")
# quantitative variables
qcovar_cols = ["FID", "IID", "p21003_i2"] + [f"p22009_a{i}" for i in range(1, 11)]
demographic_data[qcovar_cols].to_csv("ukb_qcovar.txt", sep="\t", index=False, na_rep="NA")

## Update all necesarry list

In [ ]:
# Upload GRM files to DNA Nexus
print("Uploading GRM files to DNA Nexus...")
dx_upload("final_pheno_caucasian.csv", f"/{genotype_origin_folder}/Image_dataset/final_pheno_caucasian.csv")
dx_upload("ukb_covar.txt", f"/{genotype_origin_folder}/Covariates/ukb_covar.txt")
dx_upload("ukb_qcovar.txt", f"/{genotype_origin_folder}/Covariates/ukb_qcovar.txt")